In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [2]:
# !pip install s3fs -q
# !pip install tensorflow_hub -q
# !pip install tensorflow_datasets -q
# !pip install pycocotools -q
# !pip install gin-config -q
# !pip install immutabledict -q
# !pip install sentencepiece -q
# !pip install transformers -q
# !pip install -r official/projects/movinet/requirements.txt -q

In [3]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [4]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [5]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

tf.keras.backend.clear_session()

2023-11-15 22:01:02.779088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 22:01:03.610071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
def pad_video_frames(video, n_frames=120):
  if video.shape[0] < n_frames:
    video = np.concatenate([video, 
                            np.zeros((n_frames - video.shape[0], 256, 256, 3), 
                                     dtype='float32')])
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video
  elif video.shape[0] > n_frames:
    video = video[np.round(np.linspace(0, len(video)-1, num=n_frames)).astype(int)]
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video
  else:
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video


In [8]:
video_files = pd.read_parquet("s3://asl-capstone/wlasl/video_mappings_masterfile.parquet")
video_files = video_files.drop_duplicates().reset_index(drop=True)
print(video_files.shape)
video_files.head(10)

(11938, 3)


,filepath,word,frame_count
0,asl-capstone/wlasl/RGB/viewpoint_0.npy,viewpoint,93
1,asl-capstone/wlasl/RGB/viewpoint_1.npy,viewpoint,82
2,asl-capstone/wlasl/RGB/viewpoint_2.npy,viewpoint,40
3,asl-capstone/wlasl/RGB/viewpoint_3.npy,viewpoint,81
4,asl-capstone/wlasl/RGB/viewpoint_4.npy,viewpoint,111
5,asl-capstone/wlasl/RGB/violin_0.npy,violin,105
6,asl-capstone/wlasl/RGB/violin_1.npy,violin,54
7,asl-capstone/wlasl/RGB/violin_2.npy,violin,98
8,asl-capstone/wlasl/RGB/violin_3.npy,violin,44
9,asl-capstone/wlasl/RGB/violin_4.npy,violin,85


In [17]:
# def data_generator(file_paths_df, n_frames=120):
#     for _, row in file_paths_df.iterrows():
#         with fs.open(row['path'], 'rb') as f:
#             video = np.load(f)

#         video = pad_video_frames(video, n_frames=n_frames)
#         yield video, row['caption']


# # Create TensorFlow Dataset for model input. 
# train_dataset = tf.data.Dataset.from_generator(
#     generator=lambda: data_generator(video_files, n_frames=120),
#     output_signature=(
#         tf.TensorSpec(shape=(120, 256, 256, 3), dtype=tf.float32),
#         tf.TensorSpec(shape=(), dtype=tf.string)
#     )
# )

In [ ]:
%%time

for file in tqdm(video_files['filepath'][6000:6500]):
    with fs.open(file, 'rb') as f:
        video = np.load(f)

    video = pad_video_frames(video, n_frames=120)

    new_path = file.replace('/RGB/', '/padded/')

    with fs.open(new_path, 'wb') as f:
        np.save(f, video)

131it [17:56,  8.20s/it]

In [ ]:
# %%time

# # THIS TAKES 10 MINS FOR 30 FILES
# videos = []
# # captions = []

# counter = 101
# for _, row in tqdm(video_files[101:].iterrows()):
#     with fs.open(row['path'], 'rb') as f:
#         video = np.load(f).astype(np.float32)

#     video = pad_video_frames(video, n_frames=120)
#     video = np.expand_dims(video, axis=0)
#     videos.append(video)
#     # captions += [row['caption']]

#     if counter % 50 == 0:
#         with fs.open(f"s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/padded_joined_videos_484.npy", 
#                      'wb') as f:
#             np.save(f, np.concatenate(videos, axis=0))
#         print('file saved', counter, len(videos))
#         videos = []

#     counter+=1


In [29]:
# with fs.open(f"s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/padded_joined_videos_{counter}.npy", 
#              'wb') as f:
#     np.save(f, videos[1:,:,:,:,:])